<a href="https://colab.research.google.com/github/sangjunyoo-phd/Video-Recognition-with-Gemma3-4b/blob/main/Video_Recognition_with_Gemma3_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers
from transformers import pipeline, Gemma3ForConditionalGeneration, AutoProcessor
import torch # datatype -> torch.bfloat16
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image

print(transformers.__version__)

4.50.3


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Write - colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Write - co

In [ ]:
model_name = "google/gemma-3-4b-it"

Some works need to be done in order to load 8-bit quantized gemma3:4b. The solution can be found in Here: https://github.com/google-deepmind/gemma/issues/169

https://github.com/huggingface/transformers/issues/36815

In [ ]:
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype = torch.bfloat16,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("image-text-to-text",
                model=model,
                processor=processor,
                device_map="auto")

Device set to use cuda:0


In [ ]:
# Change to the path of your video
video_path = "/content/drive/MyDrive/Colab Notebooks/WIN_20250402_21_02_19_Pro.mp4"

In [ ]:
video = cv2.VideoCapture(video_path)
frames = []
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    frames.append(frame)
video.release()

print(f"Number of Frames: {len(frames)}\tShape of Frame: {frames[0].shape}")

Number of Frames: 213	Shape of Frame: (720, 1280, 3)


In [ ]:
def sample_frames(frames, num_frames, resize=False):
  # Extract number of frames to save memory
  sampled_frames = []
  frame_indices = np.linspace(0, len(frames) - 1, num_frames, dtype=int)
  for i in frame_indices:
    frame = frames[i]
    original_height, original_width = frame.shape[:2]
    if resize: # Might be helpful to save memeory
      resized_frame = cv2.resize(frame, (original_width//2, original_height//2))
      sampled_frames.append(resized_frame)
    else:
      sampled_frames.append(frame)
  return sampled_frames

In [ ]:
sampled_frames = sample_frames(frames, 20, resize=True)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Can you briefly explain what is happening in the video?"
                }
            # Append HERE!
            # The image should be a valid URL or a path to an image file
            # Save frames to the temp_frame folder during iteration!
        ]
    }
]

In [ ]:
for i, frame in enumerate(sampled_frames):
  image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
  image.save(f"/content/frame_{i}.jpg")

  # Append the prompt
  messages[-1]["content"].append(
      {
          "type": "image",
          "image": f"/content/frame_{i}.jpg"
      }
  )

In [ ]:
output = pipe(text=messages, max_new_tokens = 200)
print(output[0]["generated_text"][-1]['content'])

The video shows a young man with dark hair and glasses, wearing a plaid shirt. He is consistently looking at the camera and smiling. He appears to be giving a friendly wave or thumbs-up in most of the shots. The background is a plain white window shade.
